# Fine Tuning Workshop

The goal of this workshop is to simply a prompt to get chatGPT to write code into a given format.

## Install Dependencies

The first step is to install dependencies

In [ ]:
pip install --upgrade openai


In [ ]:
import json
import openai
import os
import glob


## Orignal Prompt

The original prompt that we want to replace

>>>
 
According to the new user's task and our software designs listed below: 
Task: "Create a pong game".
We have decided to complete the task through a executable software with
    a static html website. As the software develoepr, 
    to satisfy the new user's demands, you should write one or multiple 
    files and make sure that every detail of the architecture is, in the end, 
    implemented as code.
Think step by step and reason yourself to the right decisions to make sure we get it right.
You will output the content of the complete code. Each file must strictly follow a 
markdown code block format, where the following tokens must be replaced such that 
"FILENAME" is the lowercase file name including the file extension, "LANGUAGE" 
is the programming language and "CODE" is the original code.  The code should have
no comments, and you must respons with only the code and file-name... nothing

[FILENAME]
```LANGUAGE
CODE
```

For example:

[index.html]
```html
<!DOCTYPE html>
<html>
  <head>
  </head>
</html>
```

You will start with the "[index.html]" file, then go to the ones that are imported by that file, and so on.
Please note that the code should be fully functional. Ensure to implement all functions.

```

## Problems with Prompt

The prompt is very long, and takes a lot of examples.  It would be better to "show" as opposed to tell.

## Example of first prompt

We will run the first prompt to see the output

In [ ]:
prompt = """
According to the new user's task and our software designs listed below: 
Task: "Create a pong game".
We have decided to complete the task through a executable software with
    a static html website. As the software develoepr, 
    to satisfy the new user's demands, you should write one or multiple 
    files and make sure that every detail of the architecture is, in the end, 
    implemented as code.
Think step by step and reason yourself to the right decisions to make sure we get it right.
You will output the content of the complete code. Each file must strictly follow a 
markdown code block format, where the following tokens must be replaced such that 
"FILENAME" is the lowercase file name including the file extension, "LANGUAGE" 
is the programming language and "CODE" is the original code.  The code should have
no comments, and you must respons with only the code and file-name... nothing

[FILENAME]
```LANGUAGE
CODE
```

For example:

[index.html]
```html
<!DOCTYPE html>
<html>
  <head>
  </head>
</html>
```

You will start with the "[index.html]" file, then go to the ones that are imported by that file, and so on.
Please note that the code should be fully functional. Ensure to implement all functions.

```

## Problems with Prompt

The prompt is very long, and takes a lot of examples.  It would be better to "show" as opposed to tell.
"""

In [ ]:
openai.api_key = os.getenv("OPENAI_API_KEY")

completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a web developer"},
    {"role": "user", "content": prompt}
  ]
)

print(completion.choices[0].message.content)

## Problems with response

The given response gives more detail than we want... we only want the code, but it gives commentary about the code... We could probably get this eliminated with prompt engineering, but we can try fine-tuning

## Prepare Example data

Example data is prepared in the `example_data` folder.  The goal is to be able to tell chat GPT to write code with a small prompt, and give data back in the following format

[filename.ext]
```code
CODE HERE
```

In [ ]:
files = glob.glob("example_data/*")
files

## Save data as a JSONL file

Here we will take the data and write it as a JSONL file

In [ ]:
system_content = "You are a developer for a company"

lines = []
for file in files:
    name = file.split("/")[-1].split(".")[0]
    user_content = name
    with open(file, 'r') as fi:
        assistant_content = json.dumps(fi.read())
    line = "{\"messages\": [{\"role\": \"system\", \"content\": \"" + system_content + "\"}, {\"role\": \"user\", \"content\": \"" + user_content + "\"}, {\"role\": \"assistant\", \"content\":"  + assistant_content + "}]}"
    lines.append(line)

In [ ]:
lines

In [ ]:
with open('developer-initialize.jsonl', 'w') as f:
    for line in lines:
        f.write(line)
        f.write("\n")


## Upload Model

While it's possible to use the API, it's easier to simply upload the model here: https://platform.openai.com/finetune

## Run the model

In [ ]:
openai.api_key = os.getenv("OPENAI_API_KEY")

completion = openai.ChatCompletion.create(
  model="ft:gpt-3.5-turbo-0613:personal::89edVQIv",
  messages=[
    {"role": "system", "content": "You are a web developer"},
    {"role": "user", "content": "Create a pong game"}
  ]
)

print(completion.choices[0].message.content)